In [2]:
!nvidia-smi

Sun Mar 27 05:36:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
url='https://raw.githubusercontent.com/sid321axn/financial_sentiment_analysis/main/finbank_data.csv'
df = pd.read_csv(url,encoding='cp1252')
from sklearn.utils import shuffle
df = shuffle(df)
df.head()

,text,label
14543,Biohit and VWR have been in close cooperation ...,positive
8768,"The decision of Tekes was based on , among oth...",neutral
10286,"In the third quarter , net sales increased by ...",positive
7257,The facility consists of a seven year bullet t...,neutral
6966,Both operating profit and turnover for the six...,positive


In [2]:
df.label.value_counts()

neutral     8951
positive    3988
negative    1841
Name: label, dtype: int64

In [3]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]

for char in spec_chars:
    df['text_clean'] = df['text'].str.replace(char,' ')

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys


,text,label,text_clean
14543,Biohit and VWR have been in close cooperation ...,positive,Biohit and VWR have been in close cooperation ...
8768,"The decision of Tekes was based on , among oth...",neutral,"The decision of Tekes was based on , among oth..."
10286,"In the third quarter , net sales increased by ...",positive,"In the third quarter , net sales increased by ..."
7257,The facility consists of a seven year bullet t...,neutral,The facility consists of a seven year bullet t...
6966,Both operating profit and turnover for the six...,positive,Both operating profit and turnover for the six...


In [4]:
df.text_clean = df.text_clean.str.lower()

In [5]:
df.head()

,text,label,text_clean
14543,Biohit and VWR have been in close cooperation ...,positive,biohit and vwr have been in close cooperation ...
8768,"The decision of Tekes was based on , among oth...",neutral,"the decision of tekes was based on , among oth..."
10286,"In the third quarter , net sales increased by ...",positive,"in the third quarter , net sales increased by ..."
7257,The facility consists of a seven year bullet t...,neutral,the facility consists of a seven year bullet t...
6966,Both operating profit and turnover for the six...,positive,both operating profit and turnover for the six...


In [6]:
categories = ['negative', 'positive','neutral']
df=df.dropna()
y = df['label'].values
X = df['text'].values

x_train, x_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.20, random_state=123)

In [7]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [10]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 428 kB/s 
     |████████████████████████████████| 22.3 MB 65.6 MB/s 
     |████████████████████████████████| 981 kB 49.8 MB/s 
     |████████████████████████████████| 263 kB 67.1 MB/s 
     |████████████████████████████████| 2.8 MB 54.7 MB/s 
     |████████████████████████████████| 1.2 MB 55.6 MB/s 
     |████████████████████████████████| 468 kB 83.3 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 53.6 MB/s 
     |████████████████████████████████| 596 kB 77.9 MB/s 
     |████████████████████████████████| 895 kB 84.3 MB/s 
  Created wheel for ktrain: filename=ktrain-0.29.3-py3-none-any.whl size=25295410 sha256=cdc975212ad6167fb7fe44004e3cf2f6ed07dc6de186a50d61c87d3e6c21b9d5
  Stored in directory: /root/.cache/pip/wheels/04/d5/2a/1a1826e16685841c68a43c31c8a892452b105f9467f64e49e6
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9931 sha256=917ff3f7b4ffe4b

In [8]:
from transformers import AutoModel
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=categories)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:422: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 22
	95percentile : 41
	99percentile : 49


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 22
	95percentile : 42
	99percentile : 49


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [9]:
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
1971/1971 [==============================] - 477s 236ms/step - loss: 0.3383 - accuracy: 0.8705 - val_loss: 0.1773 - val_accuracy: 0.9361
Epoch 2/4
1971/1971 [==============================] - 466s 236ms/step - loss: 0.1420 - accuracy: 0.9518 - val_loss: 0.1424 - val_accuracy: 0.9577
Epoch 3/4
1971/1971 [==============================] - 465s 235ms/step - loss: 0.0708 - accuracy: 0.9764 - val_loss: 0.0996 - val_accuracy: 0.9702
Epoch 4/4
1971/1971 [==============================] - 467s 236ms/step - loss: 0.0200 - accuracy: 0.9943 - val_loss: 0.1148 - val_accuracy: 0.9733


In [10]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

    negative       0.98      0.97      0.98       368
     neutral       0.98      0.98      0.98      1790
    positive       0.95      0.96      0.96       798

    accuracy                           0.97      2956
   macro avg       0.97      0.97      0.97      2956
weighted avg       0.97      0.97      0.97      2956



array([[ 358,    9,    1],
       [   5, 1749,   36],
       [   2,   26,  770]])

In [11]:
!pip install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [12]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [13]:
predictor.explain('This new partnership agreement represents a significant milestone for both parties')

In [14]:
predictor.explain('Nordea Groups operating profit increased in 2010 by 18 percent year-on-year to 3.64 billion euros and total revenue by 3 percent to 9.33 billion euros')

In [15]:
predictor.explain('The production is to be liquidated before June 2009 and 325 employees loose their jobs')

In [16]:
predictor.explain('Cerberus Capital Management LP-backed printing paper maker NewPage Corp. has posted mixed second-quarter results , casting a cloud over its planned initial public offering')